## 1) Markdown practice

### 2.2.1. This is a level 3 heading

There are also single and double hashtag titles for level 1 and 2 - but I wanted to keep my table of contents for this tutorial clean and tidy :)

You can also type some text here.

This is how you do something in **bold** or _italics_

Then you can make a list of bullet points:

* 1, 2, 3
* A, B, C
* what about an indented bullet?
  + as easy as it gets!

1. What if you want numbered lists?
2. As easy as this!

We can also add images using a few methods as long as the file is in the right path!

<img alt="alternate text goes here" src="lets-do-this.jpg" width="60%" />

![You can add some alternate text here](doh.jpg)

Finally - we can add code chunks like so to have SQL syntax highlighting turned on:

```sql
SELECT * FROM some_table
```

Or you can have code in-line like this `SELECT * FROM some_table` but the syntax highlighting won't on for these in-line code snippets.

If you want to highlight something you can also use blockquotes:

> Something you really want to emphasise

You can also do page breaks using three underscores together like below to slightly separate sections of your document

---

Finally we can also include external links to websites like [https://www.google.com]

## 2) SELECT, ORDER BY practice

What is the name of the category with the highest category_id in the dvd_rentals.category table?

In [ ]:
SELECT
name,
category_id
FROM dvd_rentals.category
ORDER BY 2 DESC
LIMIT 1;

For the films with the longest length, what is the title of the “R” rated film with the lowest replacement_cost in dvd_rentals.film table?

In [ ]:
SELECT
title,
replacement_cost,
length,
rating
FROM dvd_rentals.film
ORDER BY 3 DESC,2
LIMIT 10;

Who was the manager of the store with the highest total_sales in the dvd_rentals.sales_by_store table?

In [ ]:
SELECT
manager,
sales_by_store
FROM dvd_rentals.sales_by_store
ORDER BY 2 DESC
LIMIT 1;

What is the postal_code of the city with the 5th highest city_id in the dvd_rentals.address table?

In [ ]:
SELECT
postal_code,
city_id
FROM dvd_rentals.address
ORDER BY 2 DESC
LIMIT 5;